# TO RUN
1. Make sure you installed all the dependencies
2. open **'init_notebook.py'** and change the **projectFolder** to the src folder of the repository.
3. In the following cell change the dataDir to the data directory of the InD dataset

In [1]:
dataDir = "E:/AV/DataSetExploration/drone-dataset-tools/data/location-2/home"
# dataDir = "D:/AV/DataSetExploration/drone-dataset-tools/data/location-2/lab"
# dataDir = "/home/office/AV/DataSetExploration/data/ind/location-2/csv"

In [2]:
exec(open('init_notebook.py').read())

current working dirE:\AV\DataSetExploration\TTI-dataset-tools\src


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from tti_dataset_tools.TrajectoryTransformer import TrajectoryTransformer
from tti_dataset_tools.TrajectoryVisualizer import TrajectoryVisualizer
from tti_dataset_tools.TrajectoryCleaner import TrajectoryCleaner
from tti_dataset_tools.ColMapper import ColMapper

In [5]:
from ind_tools.IndTransformer import IndTransformer

In [6]:
sceneId = 40
# date = "2022-12-22"
date = "2022-12-09"
fps=5
minYDisplacement = 5
maxXDisplacement = 9
pedDf = pd.read_csv(os.path.join(dataDir, f"{date}-fps-{fps}-scene-{sceneId}-pedestrians.csv"))
otherDf = pd.read_csv(os.path.join(dataDir, f"{date}-fps-{fps}-scene-{sceneId}-others.csv"))
tracksMeta = pd.read_csv(os.path.join(dataDir, f"{date}-fps-{fps}-scene-{sceneId}-meta.csv"))


FileNotFoundError: [Errno 2] No such file or directory: 'E:/AV/DataSetExploration/drone-dataset-tools/data/location-2/home\\2022-12-22-fps-5-scene-40-pedestrians.csv'

In [ ]:
visualizer = TrajectoryVisualizer()
colMapper = ColMapper(
        idCol='uniqueTrackId', 
        xCol='sceneX', 
        yCol='sceneY',
        xVelCol='sceneXVelocity', 
        yVelCol='sceneYVelocity', 
        speedCol='speed',
        fps=fps,
    )
transformer = TrajectoryTransformer(colMapper)
cleaner = TrajectoryCleaner(
    colMapper = colMapper,
    minSpeed = 0.0, # we can consider them as obstacles
    maxSpeed = 3.5,
    minYDisplacement = minYDisplacement,
    maxXDisplacement = maxXDisplacement
)

indTransformer = IndTransformer(colMapper)

In [ ]:
transformer.deriveSpeed(pedDf)
transformer.deriveDisplacements(pedDf)
pedDf.describe()

In [ ]:
plt.boxplot(pedDf['displacementY'])
plt.show()
plt.boxplot(pedDf['speed'])

## Preprocessing data
### 1. Transform to local coordinate system

In [ ]:
# transformer.translateAllToLocalSource(pedDf)

# visualizer.show(pedDf, idCol = 'uniqueTrackId', xCol = 'sceneX', yCol = 'sceneY')
# visualizer.show(pedDf, idCol = 'uniqueTrackId', xCol = 'localX', yCol = 'localY')

### 2. Derive displacement in local coordinate system again.

In [ ]:
# transformer.deriveDisplacementsInLC(pedDf)

### 1. filter out data with low Y displacement

In [ ]:
cleanPedDf = cleaner.cleanByYDisplacement(pedDf, byIQR=False)
visualizer.show(pedDf, idCol = 'uniqueTrackId', xCol = 'sceneX', yCol = 'sceneY')
visualizer.show(cleanPedDf, idCol = 'uniqueTrackId', xCol = 'sceneX', yCol = 'sceneY')

### 2. Transform trajectories to local source coordinate system

In [ ]:
allPedIds = list(cleanPedDf["uniqueTrackId"].unique())
transformer.translateAllToLocalSource(cleanPedDf)
# cleanPedDf.head()

In [ ]:
# tracksMeta.head()

In [ ]:
# tracksMeta[(tracksMeta['verticalDirection'] == 'NORTH') & (tracksMeta['class'] == 'pedestrian')]

In [ ]:
# allPedIds

### 3. Rotate north-south trajectories to south-north trajectories

In [ ]:
southIds = indTransformer.convertLocalToNorth(cleanPedDf, tracksMeta)
visualizer.show(cleanPedDf, idCol = 'uniqueTrackId', xCol = 'localX', yCol = 'localY')

In [ ]:
southPeds = cleanPedDf[cleanPedDf["uniqueTrackId"].isin(southIds)]
visualizer.show(southPeds, idCol = 'uniqueTrackId', xCol = 'localX', yCol = 'localY')
northPeds = cleanPedDf[~cleanPedDf["uniqueTrackId"].isin(southIds)]
visualizer.show(northPeds, idCol = 'uniqueTrackId', xCol = 'localX', yCol = 'localY')

### 4. Shape Analysis from Local Source

### Shape dependencies:
1. Crosswalk / Midblock
2. Crosswalk - on-coming start/end, driving lane near/far
3. Crosswalk - discontinued on left/right (can distinuation point be an input?)
4. Width

### 5. Modeling approaches
1. Polygon
2. Influence heatmap (each point on a trajectory has a influence area, we need to define a datastructure for it) (or not)
3. 

#### 5.1 Polygon

In [ ]:
from tti_dataset_tools import CrosswalkModel
crosswalkModel = CrosswalkModel(colMapper)

In [ ]:
roadWidth = 6.0
interval = 1.0
yTolerance = 0.1

yBreakpoints = list(np.arange(interval, roadWidth, interval))
if (yBreakpoints[-1] - roadWidth) < 0.2:
    yBreakpoints.append(roadWidth)
    
breakpointXVals = crosswalkModel.getAllLocalXAtLocalYBreakpoints(
        tracksDf = southPeds,
        yBreakpoints = yBreakpoints,
        yTolerance = yTolerance
    )

In [ ]:
visualizer.showBreakpointVals(breakpointXVals)

In [ ]:
# with min-max
minVals = [min(breakpointXVals[y]) for y in breakpointXVals]
maxVals = [max(breakpointXVals[y]) for y in breakpointXVals]

In [ ]:
minVals

In [ ]:
maxVals

In [ ]:
polygon = crosswalkModel.generatePolygonFromBreakpointVals(breakpointXVals)

In [ ]:
polygon